# Michell Luiz Costalonga

# Módulo 11 - Exercícios

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import time

# Gráficos
import matplotlib.pyplot as plt

# Datasets
from sklearn.datasets import make_friedman1, make_classification

# Modelos de seleção
from sklearn.model_selection import cross_val_score, KFold, train_test_split

# Pre processamento
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# Algoritmos
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet

# Métricas de performance (Exercício de regressão linear)
from sklearn.metrics import mean_squared_error, r2_score

# Métricas de performance (Exercício de regressão logística)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score

# Função sigmoid
from scipy.stats import logistic

# Pipeline
from sklearn.pipeline import Pipeline

# ignorar avisos
import warnings
warnings.filterwarnings('ignore')

### Exercício: Regressão Linear:
__Parte 1:__

1- Usando a função getData(), carregue os dados disponibilizados.

In [2]:
#função para acessar os dados do exercício 1

def getData():
    X, y = make_friedman1(n_samples = 10000, n_features = 5, noise = 5.0, random_state = 0)
    return X, y

In [3]:
X, y = getData()
df = pd.DataFrame(np.c_[X, y], columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'target'])
df.head()

,x1,x2,x3,x4,x5,target
0,0.548814,0.715189,0.602763,0.544883,0.423655,15.288140
1,0.645894,0.437587,0.891773,0.963663,0.383442,21.535633
2,0.791725,0.528895,0.568045,0.925597,0.071036,15.334774
3,0.087129,0.020218,0.832620,0.778157,0.870012,18.809739
4,0.978618,0.799159,0.461479,0.780529,0.118274,16.174851


2- Separe parte dos dados para o dataset de teste.

In [4]:
dftrain, dftest = train_test_split(df, test_size = 0.25, random_state = 0)
dftrain.shape, dftest.shape

((7500, 6), (2500, 6))

In [5]:
# Função para separar as variáveis independentes da variável dependente.
def separarVar(df_): # DataFrame de entrada
    
    verificador = True
    
    # Loop para separar as variáveis independentes da variável dependente
    for item in df_.columns:
        
        if (verificador):
            
            X = df_[item].values.reshape(-1,1)
            verificador = False
            
        else:
            
            if (item != 'target'):
                X = np.c_[X, df_[item].values.reshape(-1,1)]
                
            else:
                Y = df_[item].values.reshape(-1,1)
            
    return X, Y # Dados de saída

In [6]:
Xtrain, ytrain = separarVar(dftrain)
Xtest, ytest = separarVar(dftest)

Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape

((7500, 5), (7500, 1), (2500, 5), (2500, 1))

3- Usando a metodologia de validação cruzada, teste diferentes parâmetros da regLinear - diferentes learning_rates e num_steps - para escolher a melhor combinação de parâmetros.


## Função de regressão linear implementada manualmente

Para usar a função de regressão linear abaixo, deve-se instanciá-la informando o "learning_rate" e o "num_steps". Após isso, pode-se usá-la como a função presente no scikit-learn "LinearRegression".

In [7]:
#classe regLinear para exercício

class regLinear():
    
    def __init__(self, learning_rate, num_steps):
        self.learning_rate = learning_rate
        self.num_steps = num_steps
        
    def fit(self, X, y):
        y = y.reshape(-1,1)
        m = X.shape[0] 
        k = X.shape[1] 
        theta = np.random.randn(k+1,1) 
        X_b = np.c_[np.ones((m, 1)), X] 
        for step in range(self.num_steps):
            gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
            theta = theta - self.learning_rate * gradients
        self.theta_final = theta
        #print("modelo treinado.")
        
    def predict(self, X):
        m = X.shape[0]
        X_b = np.c_[np.ones((m, 1)), X]
        preds = X_b.dot(self.theta_final)
        return preds.reshape(-1,)

## Função para realizar a validação cruzada

Função usada para realizar a validação cruzada e auxiliar na seleção do modelo e/ou parâmetros que influenciam o modelo.

### Parâmetros de entrada:

* classificador: LinearRegression(), KNearestNeighbors(n_neighbors=...), ....
* X_data: [numpy.array] Variáveis independentes 
* y_data: [numpy.array] Variável dependente
* metrica1: mean_squared_error, root_mean_squared_error, r2_score, ... Qualquer métrica que tenha como parâmetro de entrada (y_true, y_pred)
* metrica2: mean_squared_error, root_mean_squared_error, r2_score, ... Qualquer métrica que tenha como parâmetro de entrada (y_true, y_pred)
* num_folds: [int] Número de dobras (geralmente assume-se 5 ou 10).
* print_info: [Booleano] True ou False. Se True, imprimirá informações das médias das métricas para os dados de treino e validação.
* nome_metrica1: [string] Nome que aparecerá para a métrica 1.
* nome_metrica2: [string] Nome que aparecerá para a métrica 2.

### Parâmetros de saída:

* lista_metrica1_treino: [list] Lista com os valores da métrica 1 para as folds de treino
* lista_metrica1_validacao: [list] Lista com os valores da métrica 1 para as folds de validação
* lista_metrica2_treino: [list] Lista com os valores da métrica 2 para as folds de validação
* lista_metrica2_validacao: [list] Lista com os valores da métrica 2 para as folds de validação

In [8]:
def validacao_cruzada(classificador, 
                       X_data, 
                       y_data, 
                       metrica1,
                       metrica2,
                       num_folds, 
                       print_info=False, 
                       nome_metrica1='MSE',
                       nome_metrica2='R2'):
    
    lista_metrica1_treino = []
    lista_metrica2_treino = []
    lista_metrica1_validacao = []
    lista_metrica2_validacao = []
    
    kf = KFold(n_splits = num_folds)
    for train_index, val_index in kf.split(X_data, y_data):
        Xtrain_folds = X_data[train_index]
        ytrain_folds = y_data[train_index]
        Xval_fold = X_data[val_index]
        yval_fold = y_data[val_index]
        classificador.fit(Xtrain_folds, ytrain_folds)
        
        pred_treino = classificador.predict(Xtrain_folds)
        pred_validacao = classificador.predict(Xval_fold)
        
        # Guardando valores nas listas
        lista_metrica1_treino.append(metrica1(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica1_validacao.append(metrica1(y_pred = pred_validacao, y_true = yval_fold))
        lista_metrica2_treino.append(metrica2(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica2_validacao.append(metrica2(y_pred = pred_validacao, y_true = yval_fold))
        
    if print_info:
        print("Métrica: " + nome_metrica1)
        print('média treino:', np.mean(lista_metrica1_treino))
        print('média validação:', np.mean(lista_metrica1_validacao), end='\n\n')
        
        print("Métrica: " + nome_metrica2)
        print('média treino:', np.mean(lista_metrica2_treino))
        print('média validação:', np.mean(lista_metrica2_validacao), end='\n\n')
        
    return lista_metrica1_treino, lista_metrica1_validacao, lista_metrica2_treino, lista_metrica2_validacao

## Função para definir o valor mínimo e máximo de uma lista de valores

Usada para obter o valor máximo e mínimo para as métricas avaliadas

### Parâmetro de entrada:

* lista: [list] Lista com os valores das métricas obtidas usando a função "validacao_cruzada"

### Parâmetro de saída:

* minimo: Valor mínimo da métrica MSE
* maximo: Valor máximo da métrica R2

In [9]:
def minList(lista):
    minimo = np.min(np.mean(lista))
    return minimo

def maxList(lista):
    maximo = np.max(np.mean(lista))
    return maximo

## Loop para verificar quais os parâmetros ótimos (learning_rate e num_steps) para a função regLinear

In [10]:
# Início da contagem do tempo de execução
t0 = time.time()

# Faixa de valores que serão testados
#learningRate = np.linspace(0.001, 0.3, 100)
#steps = np.arange(100, 1100, 50)
learningRate = np.linspace(0.1, 0.2, 100)
steps = np.arange(400, 600, 10)

# Se print_info = False, nenhuma informação será impressa
# enquanto o modelo é testado a faixa de valores dos parâmetros 
# informados acima
print_info = False

# Listas para armazenar os valores das métricas para treino e validação,
# assim como o valor do passo e learning rate.
lt_mse_lista = []
lv_mse_lista = []
lt_r2_lista = []
lv_r2_lista = []
lista_passo = []
lista_lr = []

for step in steps:
    if print_info:
        print('\033[1mSteps: %s\033[0m' %(str(step)))
    for lr in learningRate:
        if print_info:
            print('\033[1mLearning rate: %s\033[0m' %(str(lr)))
        lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(regLinear(learning_rate = lr, num_steps = step),
                                                         X_data = Xtrain,
                                                         y_data = ytrain,
                                                         metrica1 = mean_squared_error,
                                                         metrica2 = r2_score,
                                                         num_folds = 5,
                                                         print_info = print_info)
        lt_mse_lista.append(minList(lt_mse))
        lv_mse_lista.append(minList(lv_mse))
        lt_r2_lista.append(maxList(lt_r2))
        lv_r2_lista.append(maxList(lv_r2))
        lista_passo.append(step)
        lista_lr.append(lr)

df_info_linReg = pd.DataFrame(np.c_[lista_passo, lista_lr, lt_mse_lista, lv_mse_lista, lt_r2_lista, lv_r2_lista],
                             columns=['Stpes', 'Learning Rate', 'MSE Treino', 'MSE Validação', 'R2 Treino', 'R2 Validação'])

r2_max = np.max(df_info_linReg['R2 Validação'].values)
index_max = df_info_linReg.index[df_info_linReg['R2 Validação']==r2_max][0]

# Fim da contagem do tempo de execução
t1 = time.time()
dt = t1 - t0

print('Tempo de execução: %d segundos' %(dt), end='\n\n')

highlight = lambda x:['background: lightgreen' if x['R2 Validação'] == r2_max else '' for i in x]
df_info_linReg[index_max - 2 : index_max + 3].style.apply(highlight, axis=1)

Tempo de execução: 210 segundos



,Stpes,Learning Rate,MSE Treino,MSE Validação,R2 Treino,R2 Validação
1702,570.000000,0.102020,30.828492,30.877805,0.372694,0.371467
1703,570.000000,0.103030,30.828493,30.878713,0.372694,0.371449
1704,570.000000,0.104040,30.828424,30.877624,0.372696,0.371471
1705,570.000000,0.105051,30.828386,30.878675,0.372696,0.371449
1706,570.000000,0.106061,30.828366,30.878389,0.372697,0.371455


Após rodar o primeiro loop considerando __learningRate = np.linspace(0.001, 0.3, 100)__ e __steps = np.arange(100, 1100, 50)__, os parâmetros foram alterados para __learningRate = np.linspace(0.1, 0.2, 100)__ e __steps = np.arange(400, 600, 10)__ (uma faixa mais estreita). Assim, os parâmetros ótimos foram identificados:

* learning_rate = 0.110101
* num_steps = 540

Para esses parâmetros, o valor de R$^2$ máximo obtido pelo modelo de regressão linear implementado manualmente é igual a 0.371476.

4- Implemente a regressão linear do scikit-learn e compare os resultados obtidos.

In [11]:
# Algoritmo de Regressão Linear: Scikit-Learn
lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(LinearRegression(),
                                                 X_data = Xtrain,
                                                 y_data = ytrain,
                                                 metrica1 = mean_squared_error,
                                                 metrica2 = r2_score,
                                                 num_folds = 5,
                                                 print_info = True)

Métrica: MSE
média treino: 30.82793938717482
média validação: 30.878139308128095

Métrica: R2
média treino: 0.37270542082146163
média validação: 0.3714595229612249



In [12]:
dif = ((r2_max - maxList(lv_r2))/maxList(lv_r2))*100
print('A diferença do valor de R2 entre os dois modelos é: %s %%' %(str(dif)))

A diferença do valor de R2 entre os dois modelos é: 0.003015274785161985 %


Fazendo o mesmo para o modelo de regressão linear implementado na biblioteca scikit-learn, vemos que o R$^2$ máximo obtido para a validação é de 0.371459 e que a diferença entre esse valor e o valor obtido pela regressão linear implementada manualmente é de apenas 0,004 %.

__Parte 2 (Introdução):__

Para cada variável explicativa $X_1, .., X_5$, crie outras variáveis usando o __quadrado__ de cada um delas. Desta forma, o conjunto final será de 10 variáveis, em que:

$X_6 = (X_1)^{2}$, $X_7 = (X_2)^{2}$, $X_8 = (X_3)^{2}$, $X_9 = (X_4)^{2}$, $X_{10} = (X_5)^{2}$.

Ao treinarmos uma regressão linear com essas 10 variáveis, a predição é da forma:

$y_{pred} = \theta_0 + \theta_1 \cdot X_1 + .. + \theta_5 \cdot X_5 + \theta_6 \cdot (X_1)^{2} + .. + \theta_{10} \cdot (X_5)^{2}$

Como estamos usando o quadrado das variáveis explicativas, dizemos que temos um __modelo de regressão polinomial de grau 2__. Podemos ter variações deste modelo:

-Podemos aumentar o grau: basta mudar a potência que elevamos as variáveis. Por exemplo, podemos incluir o __cubo__ das variáveis e termos um modelo polinomial de ordem 3.

-Podemos ter __interações__ entre as variáveis: multiplicações entre as variáveis.

Exemplo:

$y_{pred} = \theta_0 + \theta_1 \cdot X_1 + .. + \theta_5 \cdot X_5 + \theta_6 \cdot (X_1)^{2} + .. + \theta_{10} \cdot (X_5)^{2} + \theta_{11} \cdot (X_1)^{3} + \theta_{12} \cdot V1 + \theta_{13} \cdot V2$,

onde

$V_1 = X_1 \cdot X_2$ e $V_2 = (X_2)^{2} \cdot X_4$

__Parte 2 (Exercício):__

1- Estude o link:
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

em que é discutido como criar modelos polinomiais com o scikit-learn de forma detalhada.

2- Repita os passos da primeira parte, mas agora considerando polinômios de graus 2 ou mais.

### Criando as variáveis ao quadrado

$X_6 = (X_1)^{2}$, $X_7 = (X_2)^{2}$, $X_8 = (X_3)^{2}$, $X_9 = (X_4)^{2}$, $X_{10} = (X_5)^{2}$.

In [13]:
# Criando as variáveis Xi²
X2 = X**2

df_poli = pd.DataFrame(np.c_[X, X2, y], columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'target'])

# Dividindo os dados em treino e teste
dftrainp2, dftestp2 = train_test_split(df_poli, test_size = 0.25, random_state = 0)

# Separando as variáveis indepedentes da variável dependente
Xtrainp2, ytrainp2 = separarVar(dftrainp2)
Xtestp2, ytestp2 = separarVar(dftestp2)

Podemo checar o coeficiente de correlação entre as variáveis e ver se há interação entre as mesmas e criar novas variáveis que levem em consideração essa interação.

### Coeficiente de correlação de Pearson.

O coeficiente de correlação de Pearson pode assumir valores entre -1 e 1. Sendo que valores próximos a zero indicam que as variáveis apresentam baixa relação entre si. índices próximos a -1 indica uma correlação negativa, ou seja, a medida que uma variávem aumenta, a outra diminui. Por outro lado, para coeficientes próximos a 1, há o aumento de ambas as variáveis.

In [14]:
df_poli.corr()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,target
X1,1.000000,-0.007290,0.008929,-0.002601,-0.005767,0.968127,0.001973,0.007376,0.000321,-0.009133,0.258653
X2,-0.007290,1.000000,-0.007431,-0.000189,-0.014013,-0.005107,0.967808,-0.006895,0.003272,-0.016285,0.282831
X3,0.008929,-0.007431,1.000000,0.012058,-0.006692,0.005242,-0.007719,0.968393,0.010011,-0.003788,0.013035
X4,-0.002601,-0.000189,0.012058,1.000000,-0.000644,-0.002350,0.000048,0.011074,0.968054,-0.000320,0.431041
X5,-0.005767,-0.014013,-0.006692,-0.000644,1.000000,-0.005802,-0.012579,-0.004041,-0.005494,0.967737,0.192413
X6,0.968127,-0.005107,0.005242,-0.002350,-0.005802,1.000000,0.003045,0.003686,0.001446,-0.009312,0.218439
X7,0.001973,0.967808,-0.007719,0.000048,-0.012579,0.003045,1.000000,-0.007941,0.002682,-0.015211,0.243586
X8,0.007376,-0.006895,0.968393,0.011074,-0.004041,0.003686,-0.007941,1.000000,0.009124,-0.002143,0.068536
X9,0.000321,0.003272,0.010011,0.968054,-0.005494,0.001446,0.002682,0.009124,1.000000,-0.005745,0.419308
X10,-0.009133,-0.016285,-0.003788,-0.000320,0.967737,-0.009312,-0.015211,-0.002143,-0.005745,1.000000,0.183030


Naturalmente, as variáveis que representam o quadrado de outras, possuem alto índice de correlação com a variável "original". Por exemplo, a variável X$_6$ apresenta alta correlação com a variável X$_1$, uma vez que X$_6$ = X$_1 ^2$. De uma maneira geral, vemos que não há alta interação entre as variáveis. Aplicaremos ambos os modelos de regressão para os novos dados e verificaremos o que ocorre com o valor de R$^2$

### Loop para a regressão linear implementada manualmente

In [15]:
# Início da contagem do tempo de execução
t0 = time.time()

# Faixa de valores que serão testados
learningRate = np.linspace(0.001, 0.5, 100)
steps = np.arange(100, 1100, 100)
#learningRate = np.linspace(0.1, 0.2, 100)
#steps = np.arange(400, 600, 10)

# Se print_info = False, nenhuma informação será impressa
# enquanto o modelo é testado a faixa de valores dos parâmetros 
# informados acima
print_info = False

# Listas para armazenar os valores das métricas para treino e validação,
# assim como o valor do passo e learning rate.
lt_mse_lista = []
lv_mse_lista = []
lt_r2_lista = []
lv_r2_lista = []
lista_passo = []
lista_lr = []

for step in steps:
    if print_info:
        print('\033[1mSteps: %s\033[0m' %(str(step)))
    for lr in learningRate:
        if print_info:
            print('\033[1mLearning rate: %s\033[0m' %(str(lr)))
        lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(regLinear(learning_rate = lr, num_steps = step),
                                                         X_data = Xtrainp2,
                                                         y_data = ytrainp2,
                                                         metrica1 = mean_squared_error,
                                                         metrica2 = r2_score,
                                                         num_folds = 5,
                                                         print_info = print_info)
        lt_mse_lista.append(minList(lt_mse))
        lv_mse_lista.append(minList(lv_mse))
        lt_r2_lista.append(maxList(lt_r2))
        lv_r2_lista.append(maxList(lv_r2))
        lista_passo.append(step)
        lista_lr.append(lr)

df_info_linReg2 = pd.DataFrame(np.c_[lista_passo, lista_lr, lt_mse_lista, lv_mse_lista, lt_r2_lista, lv_r2_lista],
                             columns=['Stpes', 'Learning Rate', 'MSE Treino', 'MSE Validação', 'R2 Treino', 'R2 Validação'])

r2_max = np.max(df_info_linReg2['R2 Validação'].values)
index_max = df_info_linReg2.index[df_info_linReg2['R2 Validação']==r2_max][0]

# Fim da contagem do tempo de execução
t1 = time.time()
dt = t1 - t0

print('Tempo de execução: %d segundos' %(dt), end='\n\n')

highlight = lambda x:['background: lightgreen' if x['R2 Validação'] == r2_max else '' for i in x]
df_info_linReg2[index_max - 2 : index_max + 3].style.apply(highlight, axis=1)

Tempo de execução: 115 segundos



,Stpes,Learning Rate,MSE Treino,MSE Validação,R2 Treino,R2 Validação
965,1000.000000,0.328626,26.792142,26.862827,0.454821,0.453127
966,1000.000000,0.333667,26.791021,26.864937,0.454844,0.453082
967,1000.000000,0.338707,26.773247,26.845596,0.455207,0.453471
968,1000.000000,0.343747,37.082318,36.843817,0.246443,0.246014
969,1000.000000,0.348788,364835279699865670390906880.000000,359177910954036856107302912.000000,-7388064902848526659092480.000000,-7452784995666652823552000.000000


In [16]:
# Início da contagem do tempo de execução
t0 = time.time()

# Faixa de valores que serão testados
#learningRate = np.linspace(0.001, 0.5, 100)
#steps = np.arange(100, 1100, 100)
learningRate = np.linspace(0.3, 0.4, 100)
steps = np.arange(1000, 1200, 10)

# Se print_info = False, nenhuma informação será impressa
# enquanto o modelo é testado a faixa de valores dos parâmetros 
# informados acima
print_info = False

# Listas para armazenar os valores das métricas para treino e validação,
# assim como o valor do passo e learning rate.
lt_mse_lista = []
lv_mse_lista = []
lt_r2_lista = []
lv_r2_lista = []
lista_passo = []
lista_lr = []

for step in steps:
    if print_info:
        print('\033[1mSteps: %s\033[0m' %(str(step)))
    for lr in learningRate:
        if print_info:
            print('\033[1mLearning rate: %s\033[0m' %(str(lr)))
        lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(regLinear(learning_rate = lr, num_steps = step),
                                                         X_data = Xtrainp2,
                                                         y_data = ytrainp2,
                                                         metrica1 = mean_squared_error,
                                                         metrica2 = r2_score,
                                                         num_folds = 5,
                                                         print_info = print_info)
        lt_mse_lista.append(minList(lt_mse))
        lv_mse_lista.append(minList(lv_mse))
        lt_r2_lista.append(maxList(lt_r2))
        lv_r2_lista.append(maxList(lv_r2))
        lista_passo.append(step)
        lista_lr.append(lr)

df_info_linReg2 = pd.DataFrame(np.c_[lista_passo, lista_lr, lt_mse_lista, lv_mse_lista, lt_r2_lista, lv_r2_lista],
                             columns=['Stpes', 'Learning Rate', 'MSE Treino', 'MSE Validação', 'R2 Treino', 'R2 Validação'])

r2_max = np.max(df_info_linReg2['R2 Validação'].values)
index_max = df_info_linReg2.index[df_info_linReg2['R2 Validação']==r2_max][0]

# Fim da contagem do tempo de execução
t1 = time.time()
dt = t1 - t0

print('Tempo de execução: %d segundos' %(dt), end='\n\n')

highlight = lambda x:['background: lightgreen' if x['R2 Validação'] == r2_max else '' for i in x]
df_info_linReg2[index_max - 2 : index_max + 3].style.apply(highlight, axis=1)

Tempo de execução: 448 segundos



,Stpes,Learning Rate,MSE Treino,MSE Validação,R2 Treino,R2 Validação
1840,1180.000000,0.340404,26.712852,26.791055,0.456435,0.454581
1841,1180.000000,0.341414,26.714901,26.790452,0.456393,0.454592
1842,1180.000000,0.342424,26.710090,26.786448,0.456490,0.454674
1843,1180.000000,0.343434,26.976915,27.021384,0.451087,0.449799
1844,1180.000000,0.344444,194502.431234,191462.757047,-3937.755763,-3971.763407


In [17]:
# Algoritmo de Regressão Linear: Scikit-Learn
lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(LinearRegression(),
                                                 X_data = Xtrainp2,
                                                 y_data = ytrainp2,
                                                 metrica1 = mean_squared_error,
                                                 metrica2 = r2_score,
                                                 num_folds = 5,
                                                 print_info = True)

Métrica: MSE
média treino: 26.650126278115778
média validação: 26.729533589704182

Métrica: R2
média treino: 0.45771105520472516
média validação: 0.4558181308190643



In [18]:
dif = ((r2_max - maxList(lv_r2))/maxList(lv_r2))*100
print('A diferença do valor de R2 entre os dois modelos é: %s %%' %(str(dif)))

A diferença do valor de R2 entre os dois modelos é: -0.25094416464027863 %


Nesse caso, a regressão linear implementada manualmente apresenta um R$^2$ com valor inferior a 0,24 %, quando comparada à LinearRegression da biblioteca scikit-learn.

Usaremos agora a função PolynomialFeatures para obter as novas variáveis, assim como suas interações.

In [19]:
df_x = pd.DataFrame(np.c_[X], columns=['X1','X2','X3','X4','X5'])
poly = PolynomialFeatures(degree=2, include_bias=False)
x2 = poly.fit_transform(df_x)

nomes = poly.get_feature_names(df_x.columns)
nomes.append('target')
nomes

df_ = pd.DataFrame(np.c_[x2,y], columns=[nomes])

df_

,X1,X2,X3,X4,X5,X1^2,X1 X2,X1 X3,X1 X4,X1 X5,...,X2 X3,X2 X4,X2 X5,X3^2,X3 X4,X3 X5,X4^2,X4 X5,X5^2,target
0,0.548814,0.715189,0.602763,0.544883,0.423655,0.301196,0.392506,0.330805,0.299039,0.232507,...,0.431090,0.389695,0.302993,0.363324,0.328436,0.255364,0.296898,0.230842,0.179483,15.288140
1,0.645894,0.437587,0.891773,0.963663,0.383442,0.417179,0.282635,0.575991,0.622424,0.247663,...,0.390228,0.421686,0.167789,0.795259,0.859368,0.341943,0.928646,0.369508,0.147027,21.535633
2,0.791725,0.528895,0.568045,0.925597,0.071036,0.626829,0.418739,0.449735,0.732818,0.056241,...,0.300436,0.489543,0.037571,0.322675,0.525780,0.040352,0.856729,0.065751,0.005046,15.334774
3,0.087129,0.020218,0.832620,0.778157,0.870012,0.007592,0.001762,0.072546,0.067800,0.075804,...,0.016834,0.015733,0.017590,0.693256,0.647909,0.724389,0.605528,0.677006,0.756921,18.809739
4,0.978618,0.799159,0.461479,0.780529,0.118274,0.957694,0.782071,0.451612,0.763840,0.115746,...,0.368795,0.623767,0.094520,0.212963,0.360198,0.054581,0.609226,0.092317,0.013989,16.174851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.354060,0.656584,0.538141,0.397419,0.861492,0.125359,0.232470,0.190534,0.140710,0.305020,...,0.353334,0.260939,0.565642,0.289595,0.213867,0.463604,0.157942,0.342373,0.742169,6.827949
9996,0.107033,0.039126,0.687082,0.402869,0.083409,0.011456,0.004188,0.073540,0.043120,0.008928,...,0.026883,0.015763,0.003263,0.472082,0.276804,0.057309,0.162304,0.033603,0.006957,4.256359
9997,0.255888,0.191853,0.318892,0.970987,0.458267,0.065479,0.049093,0.081601,0.248464,0.117265,...,0.061180,0.186286,0.087920,0.101692,0.309640,0.146138,0.942815,0.444972,0.210009,13.554278
9998,0.650715,0.877337,0.602423,0.426482,0.655736,0.423430,0.570896,0.392006,0.277518,0.426697,...,0.528528,0.374169,0.575302,0.362914,0.256923,0.395031,0.181887,0.279660,0.429990,20.053885


Aplicaremos, novamente, a regressão linear a esses dados. Sendo assim, separaremos os dados de treino e de teste.

In [20]:
# Dividindo os dados em treino e teste
dftrainp2, dftestp2 = train_test_split(df_poli, test_size = 0.25, random_state = 0)

# Separando as variáveis indepedentes da variável dependente
Xtrainp2, ytrainp2 = separarVar(dftrainp2)
Xtestp2, ytestp2 = separarVar(dftestp2)

In [21]:
# Início da contagem do tempo de execução
t0 = time.time()

# Faixa de valores que serão testados
learningRate = np.linspace(0.001, 0.5, 100)
steps = np.arange(100, 1100, 100)
#learningRate = np.linspace(0.1, 0.2, 100)
#steps = np.arange(400, 600, 10)

# Se print_info = False, nenhuma informação será impressa
# enquanto o modelo é testado a faixa de valores dos parâmetros 
# informados acima
print_info = False

# Listas para armazenar os valores das métricas para treino e validação,
# assim como o valor do passo e learning rate.
lt_mse_lista = []
lv_mse_lista = []
lt_r2_lista = []
lv_r2_lista = []
lista_passo = []
lista_lr = []

for step in steps:
    if print_info:
        print('\033[1mSteps: %s\033[0m' %(str(step)))
    for lr in learningRate:
        if print_info:
            print('\033[1mLearning rate: %s\033[0m' %(str(lr)))
        lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(regLinear(learning_rate = lr, num_steps = step),
                                                         X_data = Xtrainp2,
                                                         y_data = ytrainp2,
                                                         metrica1 = mean_squared_error,
                                                         metrica2 = r2_score,
                                                         num_folds = 5,
                                                         print_info = print_info)
        lt_mse_lista.append(minList(lt_mse))
        lv_mse_lista.append(minList(lv_mse))
        lt_r2_lista.append(maxList(lt_r2))
        lv_r2_lista.append(maxList(lv_r2))
        lista_passo.append(step)
        lista_lr.append(lr)

df_info_linReg2 = pd.DataFrame(np.c_[lista_passo, lista_lr, lt_mse_lista, lv_mse_lista, lt_r2_lista, lv_r2_lista],
                             columns=['Stpes', 'Learning Rate', 'MSE Treino', 'MSE Validação', 'R2 Treino', 'R2 Validação'])

r2_max = np.max(df_info_linReg2['R2 Validação'].values)
index_max = df_info_linReg2.index[df_info_linReg2['R2 Validação']==r2_max][0]

# Fim da contagem do tempo de execução
t1 = time.time()
dt = t1 - t0

print('Tempo de execução: %d segundos' %(dt), end='\n\n')

highlight = lambda x:['background: lightgreen' if x['R2 Validação'] == r2_max else '' for i in x]
df_info_linReg2[index_max - 2 : index_max + 3].style.apply(highlight, axis=1)

Tempo de execução: 116 segundos



,Stpes,Learning Rate,MSE Treino,MSE Validação,R2 Treino,R2 Validação
965,1000.000000,0.328626,26.794107,26.871850,0.454782,0.452939
966,1000.000000,0.333667,26.784322,26.861933,0.454980,0.453144
967,1000.000000,0.338707,26.780733,26.854295,0.455053,0.453300
968,1000.000000,0.343747,53.687010,53.089671,-0.089809,-0.091081
969,1000.000000,0.348788,283922025708835973156044800.000000,279519401541613715518914560.000000,-5749539103508913987780608.000000,-5799901918925171437076480.000000


In [22]:
# Início da contagem do tempo de execução
t0 = time.time()

# Faixa de valores que serão testados
#learningRate = np.linspace(0.001, 0.5, 100)
#steps = np.arange(100, 1100, 100)
learningRate = np.linspace(0.3, 0.4, 100)
steps = np.arange(900, 1100, 25)

# Se print_info = False, nenhuma informação será impressa
# enquanto o modelo é testado a faixa de valores dos parâmetros 
# informados acima
print_info = False

# Listas para armazenar os valores das métricas para treino e validação,
# assim como o valor do passo e learning rate.
lt_mse_lista = []
lv_mse_lista = []
lt_r2_lista = []
lv_r2_lista = []
lista_passo = []
lista_lr = []

for step in steps:
    if print_info:
        print('\033[1mSteps: %s\033[0m' %(str(step)))
    for lr in learningRate:
        if print_info:
            print('\033[1mLearning rate: %s\033[0m' %(str(lr)))
        lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(regLinear(learning_rate = lr, num_steps = step),
                                                         X_data = Xtrainp2,
                                                         y_data = ytrainp2,
                                                         metrica1 = mean_squared_error,
                                                         metrica2 = r2_score,
                                                         num_folds = 5,
                                                         print_info = print_info)
        lt_mse_lista.append(minList(lt_mse))
        lv_mse_lista.append(minList(lv_mse))
        lt_r2_lista.append(maxList(lt_r2))
        lv_r2_lista.append(maxList(lv_r2))
        lista_passo.append(step)
        lista_lr.append(lr)

df_info_linReg2 = pd.DataFrame(np.c_[lista_passo, lista_lr, lt_mse_lista, lv_mse_lista, lt_r2_lista, lv_r2_lista],
                             columns=['Stpes', 'Learning Rate', 'MSE Treino', 'MSE Validação', 'R2 Treino', 'R2 Validação'])

r2_max = np.max(df_info_linReg2['R2 Validação'].values)
index_max = df_info_linReg2.index[df_info_linReg2['R2 Validação']==r2_max][0]

# Fim da contagem do tempo de execução
t1 = time.time()
dt = t1 - t0

print('Tempo de execução: %d segundos' %(dt), end='\n\n')

highlight = lambda x:['background: lightgreen' if x['R2 Validação'] == r2_max else '' for i in x]
df_info_linReg2[index_max - 2 : index_max + 3].style.apply(highlight, axis=1)

Tempo de execução: 162 segundos



,Stpes,Learning Rate,MSE Treino,MSE Validação,R2 Treino,R2 Validação
739,1075.000000,0.339394,26.747167,26.826553,0.455736,0.453861
740,1075.000000,0.340404,26.742788,26.820150,0.455826,0.453988
741,1075.000000,0.341414,26.740452,26.816924,0.455873,0.454055
742,1075.000000,0.342424,26.742183,26.817565,0.455838,0.454044
743,1075.000000,0.343434,27.196626,27.300173,0.446635,0.444033


In [23]:
# Algoritmo de Regressão Linear: Scikit-Learn
lt_mse, lv_mse, lt_r2, lv_r2 = validacao_cruzada(LinearRegression(),
                                                 X_data = Xtrainp2,
                                                 y_data = ytrainp2,
                                                 metrica1 = mean_squared_error,
                                                 metrica2 = r2_score,
                                                 num_folds = 5,
                                                 print_info = True)

Métrica: MSE
média treino: 26.650126278115778
média validação: 26.729533589704182

Métrica: R2
média treino: 0.45771105520472516
média validação: 0.4558181308190643



In [24]:
dif = ((r2_max - maxList(lv_r2))/maxList(lv_r2))*100
print('A diferença do valor de R2 entre os dois modelos é: %s %%' %(str(dif)))

A diferença do valor de R2 entre os dois modelos é: -0.38678801617041014 %


Assim como no caso anteior, vemos uma piora quando comparado ao modelo implementado no scikit-learn.

3- Inclua regularização Ridge e Lasso nas análises e teste os resultados para diferentes parâmetros $\alpha$.

### Incluindo a regularização de Ridge, Lasso e ElasticNet

Considerando um polinômio de grau dois, serão aplicadas as regularizações L1: Lasso e L2: Ridge. Em ambos os casos, o efeito da regularização é diminuir os pesos do somatório de $\theta$, e penalizar grandes pesos. Porém, a maneira como os pesos diminuem é diferente. Na Regularização L1, os pesos diminuem em uma quantidade constante para 0. Na Regularização L2, os pesos diminuem em um valor proporcional a soma do quadrado de $\theta$. 

Com isso em mente, quando um custo tem uma grande magnitude, a Regularização L1 reduz o MSE muito menos do que a Regularização L2. Em contraste, quando o custo é pequeno, a Regularização L1 reduz o peso muito mais do que a Regularização L2. O resultado é que a Regularização L1 tende a concentrar o peso da rede em um número relativamente pequeno de conexões de alta importância, enquanto os outros pesos são direcionados para zero.

#### Regularização Ridge

A função de custo é obtida pela seguinte equação:

#### $Custo(\theta) = MSE(\theta) + \alpha \cdot \sum {\theta_i}^2$, 
em que a soma $\sum {\theta_i}^2$ é para valores de $i=1, .., p$.


#### Regularização Lasso

Neste caso, a função de custo a ser minimizada é:

#### $Custo(\theta) = MSE(\theta) + \alpha \cdot \sum |{\theta_i}|$, 
em que a soma $\sum |\theta_i|$ é para valores de $i=1, .., p$.

In [25]:
def polyFitReg(X, y, grau, base_model, base_model_name):
    
    polybig_features = PolynomialFeatures(degree = grau, include_bias = False)
    std_scaler = StandardScaler()
    basemodel = base_model
    
    #criando um pipeline: sequencia de execução de passos
    polynomial_regression = Pipeline([("poly_features", polybig_features),
                                      ("std_scaler", std_scaler),
                                      (base_model_name, base_model),])
    
    polynomial_regression.fit(X, y)
    return polynomial_regression

In [26]:
grau = 2 

for alpha in [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]:
    
    model_name = 'Ridge - alpha: ' + str(alpha)
    polyfit = polyFitReg(Xtrain, 
                         ytrain, 
                         grau, 
                         base_model = Ridge(alpha = alpha), 
                         base_model_name = model_name)

    print(model_name)
    print('R2:')
    print('Treino:', r2_score(y_true = ytrain, y_pred = polyfit.predict(Xtrain)))
    print('Teste:', r2_score(y_true = ytest, y_pred = polyfit.predict(Xtest)))
    
    print('MSE:')
    print('Treino:', mean_squared_error(y_true = ytrain, y_pred = polyfit.predict(Xtrain)))
    print('Teste:', mean_squared_error(y_true = ytest, y_pred = polyfit.predict(Xtest)))
    print("------------------------------------------------\n\n")

Ridge - alpha: 0
R2:
Treino: 0.45848057470688686
Teste: 0.4564242863320783
MSE:
Treino: 26.613220274804387
Teste: 26.806401868338195
------------------------------------------------


Ridge - alpha: 0.001
R2:
Treino: 0.4584805747045849
Teste: 0.45642429586713884
MSE:
Treino: 26.613220274917516
Teste: 26.806401398117295
------------------------------------------------


Ridge - alpha: 0.01
R2:
Treino: 0.45848057447671464
Teste: 0.4564243814812976
MSE:
Treino: 26.613220286116306
Teste: 26.806397176060475
------------------------------------------------


Ridge - alpha: 0.1
R2:
Treino: 0.45848055171136737
Teste: 0.4564252177062792
MSE:
Treino: 26.61322140492973
Teste: 26.80635593767851
------------------------------------------------


Ridge - alpha: 1
R2:
Treino: 0.4584782966862516
Teste: 0.45643160868719157
MSE:
Treino: 26.613332229172272
Teste: 26.80604076685987
------------------------------------------------


Ridge - alpha: 10
R2:
Treino: 0.45827271177455464
Teste: 0.456317271319111

In [27]:
grau = 2 

for alpha in [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]:
    
    model_name = 'Lasso - alpha: ' + str(alpha)
    polyfit = polyFitReg(Xtrain, 
                         ytrain, 
                         grau, 
                         base_model = Lasso(alpha = alpha), 
                         base_model_name = model_name)

    print(model_name)
    print('R2:')
    print('Treino:', r2_score(y_true = ytrain, y_pred = polyfit.predict(Xtrain)))
    print('Teste:', r2_score(y_true = ytest, y_pred = polyfit.predict(Xtest)))
    
    print('MSE:')
    print('Treino:', mean_squared_error(y_true = ytrain, y_pred = polyfit.predict(Xtrain)))
    print('Teste:', mean_squared_error(y_true = ytest, y_pred = polyfit.predict(Xtest)))
    print("------------------------------------------------\n\n")

Lasso - alpha: 0
R2:
Treino: 0.45848057470688686
Teste: 0.4564242863320783
MSE:
Treino: 26.613220274804387
Teste: 26.806401868338195
------------------------------------------------


Lasso - alpha: 0.001
R2:
Treino: 0.4584724519902219
Teste: 0.45645944207205547
MSE:
Treino: 26.613619469436635
Teste: 26.804668165249353
------------------------------------------------


Lasso - alpha: 0.01
R2:
Treino: 0.45797275984796815
Teste: 0.4561551517620106
MSE:
Treino: 26.638177068721628
Teste: 26.81967422260365
------------------------------------------------


Lasso - alpha: 0.1
R2:
Treino: 0.41815901941737454
Teste: 0.4177077959480122
MSE:
Treino: 28.59484158444006
Teste: 28.715703137822366
------------------------------------------------


Lasso - alpha: 1
R2:
Treino: 0.30921503524400196
Teste: 0.3115624496576289
MSE:
Treino: 33.948943603682324
Teste: 33.95025416276425
------------------------------------------------


Lasso - alpha: 10
R2:
Treino: 0.0
Teste: -4.102890728940878e-06
MSE:
Trein

Como o MSE para o caso acima é relativamente pequeno, vemos que o modelo responde de maneira mais sensível à regularização L1, sendo que para ambos os casos, o aumento de $\alpha$ provoca o underfitting do modelo, prejudicando a métrica R$^2$, assim como a MSE.

Considerando ambos os modelos de regularização, temos a Elastic Net. Considerando alpha = 1 para ambos os modelos, podemos ver a influência do peso de cada regularização ao alterar o parâmetro "l1_ratio". Isso é, $ 0 \leq $ l1_ratio $\leq 1$. Se l1_ratio = 1, o peso da regularização L2 é zero, sendo assim, o modelo responderá igual ao Lasso. De forma semelhante, quando l1_ratio = 0, o modelo responderá igual ao modelo Ridge. Para o teste, consideraremos alpha = 1 e será variado o l1_ratio.

In [28]:
grau = 2 
for alpha1 in [0, 0.001, 0.01, 0.1, 0.2, 0.5, 0.7, 1]:

    model_name = 'ElasticNet - l1_ratio: ' + str(alpha1)
    polyfit = polyFitReg(Xtrain, 
                         ytrain, 
                         grau, 
                         base_model = ElasticNet(alpha = 0.01, l1_ratio = alpha1), 
                         base_model_name = model_name)

    print(model_name)
    print('R2:')
    print('Treino:', r2_score(y_true = ytrain, y_pred = polyfit.predict(Xtrain)))
    print('Teste:', r2_score(y_true = ytest, y_pred = polyfit.predict(Xtest)))

    print('MSE:')
    print('Treino:', mean_squared_error(y_true = ytrain, y_pred = polyfit.predict(Xtrain)))
    print('Teste:', mean_squared_error(y_true = ytest, y_pred = polyfit.predict(Xtest)))
    print("------------------------------------------------\n\n")

ElasticNet - l1_ratio: 0
R2:
Treino: 0.4517782666984409
Teste: 0.45055324629058247
MSE:
Treino: 26.942608272809835
Teste: 27.095931835884887
------------------------------------------------


ElasticNet - l1_ratio: 0.001
R2:
Treino: 0.45178573725884064
Teste: 0.4505601239341315
MSE:
Treino: 26.94224112869605
Teste: 27.095592665329168
------------------------------------------------


ElasticNet - l1_ratio: 0.01
R2:
Treino: 0.4518529635727183
Teste: 0.4506219981518734
MSE:
Treino: 26.93893726069811
Teste: 27.092541342202736
------------------------------------------------


ElasticNet - l1_ratio: 0.1
R2:
Treino: 0.45252403878499625
Teste: 0.4512382843066075
MSE:
Treino: 26.90595696191077
Teste: 27.062149229541525
------------------------------------------------


ElasticNet - l1_ratio: 0.2
R2:
Treino: 0.4532645051359371
Teste: 0.45191516932170894
MSE:
Treino: 26.869566403819526
Teste: 27.02876868792198
------------------------------------------------


ElasticNet - l1_ratio: 0.5
R2:
Tre

Vemos que a resposta para l1_ratio = 1 e l1_ratio = 0 são semelhantes às obtidas para os modelos Lasso e Ridge, respectivamente.

### Exercício: Regressão Logística:

__Parte 1:__

Crie uma classe regLogistica para treinar o modelo de regressão logística. Essa classe deve ser usada para problemas de classificação binária, cuja variável target assume os valores: 0 (classe negativa) e 1 (classe positiva).

O método construtor dessa classe deve possuir 3 parâmetros: learning_rate, num_steps e limiar.

Os outros médotos devem ser:

    - médoto fit: para treinar o modelo - usando gradient descent
    
    - médoto predict_proba: para retornar a probabilidade da classe 1
    
    - médoto predict: retornar a classe predita: 0 ou 1 - dependente do limiar

In [29]:
class regLogistica():
    
    def __init__(self, learning_rate, num_steps, limiar = 0.5):
        self.learning_rate = learning_rate
        self.num_steps = num_steps
        self.limiar = limiar
        
    def fit(self, X, y):
        y = y.reshape(-1,1)
        m = X.shape[0] 
        k = X.shape[1] 
        theta = np.random.randn(k+1,1) 
        X_b = np.c_[np.ones((m, 1)), X] 
        for step in range(self.num_steps):
            gradients = X_b.T.dot(logistic.cdf(X_b.dot(theta)) - y)
            theta = theta - self.learning_rate * gradients
        self.theta_final = theta
        #print("modelo treinado.")
        
    def predict(self, X):
        m = X.shape[0]
        X_b = np.c_[np.ones((m, 1)), X]
        preds = X_b.dot(self.theta_final)
        preds = np.where(preds > self.limiar, 1, 0)
        return preds.reshape(-1,)
    
    # Retorna apenas a probabilidade da classe positiva, ou seja, um array com N linhas e uma única coluna.
    def predict_proba(self, X): 
        m = X.shape[0]
        X_b = np.c_[np.ones((m, 1)), X]
        preds_proba = logistic.cdf(X_b.dot(self.theta_final))
        return preds_proba.reshape(-1,)

__Parte 2:__

Usando a função getData2(), carregue o dataset disponibilizado.

Use a regLogistica, classe criada na parte 1 do exercício, para treinar modelos nestes dados. Use validação cruzada para seleção dos parâmetros. Considere diferentes métricas de classificação e justifique as escolhas.

In [30]:
#função para acessar os dados do exercício 2

def getData2():
    X, y = make_classification(n_classes=2, n_features=5, n_samples=10000, random_state = 0)
    return X, y

Xa, ya = getData2()

# Criando dataframe
dfa = pd.DataFrame(np.c_[Xa, ya], columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'target'])

# Separando: dados de treino/validação e dados de teste
dftrain, dftest = train_test_split(dfa, test_size = 0.25, random_state = 0)

# Separando: Variáveis independentes de variáveis dependentes (treino/validação e teste)
Xtrain, ytrain = separarVar(dftrain)
Xtest, ytest = separarVar(dftest)

Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape

((7500, 5), (7500, 1), (2500, 5), (2500, 1))

In [31]:
def validacao_cruzada_rl(classificador, 
                         X_data, 
                         y_data, 
                         metrica1=precision_score,
                         metrica2=recall_score,
                         metrica3=f1_score,
                         metrica4=roc_auc_score,
                         num_folds=5, 
                         print_info=False, 
                         nome_metrica1='Precision',
                         nome_metrica2='Recall',
                         nome_metrica3='F1 Score',
                         nome_metrica4='AUC-ROC'):
    
    lista_metrica1_treino = []
    lista_metrica2_treino = []
    lista_metrica3_treino = []
    lista_metrica4_treino = []
    lista_metrica1_validacao = []
    lista_metrica2_validacao = []
    lista_metrica3_validacao = []
    lista_metrica4_validacao = []
    
    kf = KFold(n_splits = num_folds)
    for train_index, val_index in kf.split(X_data, y_data):
        Xtrain_folds = X_data[train_index]
        ytrain_folds = y_data[train_index]
        Xval_fold = X_data[val_index]
        yval_fold = y_data[val_index]
        classificador.fit(Xtrain_folds, ytrain_folds)
        
        pred_treino = classificador.predict(Xtrain_folds)
        pred_treino_proba = classificador.predict_proba(Xtrain_folds)
        
        pred_validacao = classificador.predict(Xval_fold)
        pred_validacao_proba = classificador.predict_proba(Xval_fold)
        
        # Guardando valores nas listas
        lista_metrica1_treino.append(metrica1(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica1_validacao.append(metrica1(y_pred = pred_validacao, y_true = yval_fold))
        
        lista_metrica2_treino.append(metrica2(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica2_validacao.append(metrica2(y_pred = pred_validacao, y_true = yval_fold))
        
        lista_metrica3_treino.append(metrica3(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica3_validacao.append(metrica3(y_pred = pred_validacao, y_true = yval_fold))
        
        lista_metrica4_treino.append(metrica4(y_score = pred_treino_proba, y_true = ytrain_folds))
        lista_metrica4_validacao.append(metrica4(y_score = pred_validacao_proba, y_true = yval_fold))
        
    if print_info:
        print("Métrica: " + nome_metrica1)
        print('média treino:', np.mean(lista_metrica1_treino))
        print('média validação:', np.mean(lista_metrica1_validacao), end='\n\n')
        
        print("Métrica: " + nome_metrica2)
        print('média treino:', np.mean(lista_metrica2_treino))
        print('média validação:', np.mean(lista_metrica2_validacao), end='\n\n')
        
        print("Métrica: " + nome_metrica3)
        print('média treino:', np.mean(lista_metrica3_treino))
        print('média validação:', np.mean(lista_metrica3_validacao), end='\n\n')
        
        print("Métrica: " + nome_metrica4)
        print('média treino:', np.mean(lista_metrica4_treino))
        print('média validação:', np.mean(lista_metrica4_validacao), end='\n\n')
    
    lista_metrica1_treino = np.mean(lista_metrica1_treino)
    lista_metrica2_treino = np.mean(lista_metrica2_treino)
    lista_metrica3_treino = np.mean(lista_metrica3_treino)
    lista_metrica4_treino = np.mean(lista_metrica4_treino)
    
    lista_metrica1_validacao = np.mean(lista_metrica1_validacao)
    lista_metrica2_validacao = np.mean(lista_metrica2_validacao)
    lista_metrica3_validacao = np.mean(lista_metrica3_validacao)
    lista_metrica4_validacao = np.mean(lista_metrica4_validacao)
    
    df_ = pd.DataFrame(np.c_[lista_metrica1_treino, lista_metrica1_validacao, lista_metrica2_treino, lista_metrica2_validacao, lista_metrica3_treino, lista_metrica3_validacao, lista_metrica4_treino, lista_metrica4_validacao],
                      columns=['Precision (T)', 'Precision (V)', 'Recall (T)', 'Recall (V)', 'F1 (T)', 'F1 (V)', 'ROC-AUC (T)', 'ROC-AUC (V)'])
    if print_info:
        display(df_)
    
    return df_

In [32]:
# Início da contagem do tempo de execução
t0 = time.time()

# Imprimir informações durante a execução
print_info = False

# Listas para armazenar métricas, learning_rate, num_steps e limiar
df_metricas = []
df_lr = []
df_ns = []
df_limiar = []

# Contador
i = 0

# Faixa de valores que serão testados
learningRate = np.arange(0.01, 0.041, 0.001)
steps = np.arange(80, 210, 10)
limiares = np.arange(0.3, 0.75, 0.05)

for step in steps:
    for lr in learningRate:
        for limiar in limiares:
            dfb = validacao_cruzada_rl(classificador = regLogistica(learning_rate = lr, num_steps = step, limiar = limiar),
                                       X_data = Xtrain,
                                       y_data = ytrain,
                                       num_folds = 5,
                                       print_info=print_info)
            if (i == 0):
                df_metricas = dfb.copy()
            else:
                df_metricas.loc[i] = dfb.loc[:].values[0]
            
            df_lr.append(lr)
            df_ns.append(step)
            df_limiar.append(limiar)
            i += 1
            
nomes_colunas = ['Num. Passos', 'Learning Rate', 'Threshold']
nomes_colunas.extend(df_metricas.columns)

df_proc = pd.DataFrame(np.c_[df_ns, df_lr, df_limiar, df_metricas], columns=nomes_colunas)

f1_max = np.max(df_proc['F1 (V)'].values)
index_max = df_proc.index[df_proc['F1 (V)'] == f1_max][0]

# Fim da contagem do tempo de execução
t1 = time.time()
dt = t1 - t0

print('Tempo de execução: %d segundos' %(dt), end='\n\n')

highlight = lambda x:['background: lightgreen' if x['F1 (V)'] == f1_max else '' for i in x]
df_proc[index_max - 2 : index_max + 3].style.apply(highlight, axis=1)

Tempo de execução: 1044 segundos



,Num. Passos,Learning Rate,Threshold,Precision (T),Precision (V),Recall (T),Recall (V),F1 (T),F1 (V),ROC-AUC (T),ROC-AUC (V)
2720,170.000000,0.033000,0.400000,0.836245,0.835921,0.795721,0.795851,0.815191,0.814969,0.884804,0.884556
2721,170.000000,0.033000,0.450000,0.828319,0.827894,0.810918,0.811133,0.819404,0.819124,0.882167,0.880298
2722,170.000000,0.033000,0.500000,0.811608,0.811445,0.843065,0.843594,0.827011,0.826904,0.868097,0.867449
2723,170.000000,0.033000,0.550000,0.687273,0.687550,0.697855,0.697959,0.691298,0.691547,0.718765,0.721083
2724,170.000000,0.033000,0.600000,0.696435,0.695645,0.685343,0.683463,0.690705,0.689086,0.725734,0.722638


Nesse caso, escolheremos a métrica F1 Score, uma vez que ela é uma média harmônica entre a Precision e a Recall, ponderando, assim, os falsos negativos e falsos positivos. Dessa forma, buscamos os valores de num_steps, learning_rate e limiar que maximiza o valor de F1 (V), no caso da regLogisitica. Esse valores são mostrados na linha verde em destaque na tabela acima, nas três primeiras colunas.

Podemos comparar o valor do F1 score médio obtido para a validação acima com o modelo de Regressão Logística implementado no Scikit-Learn.

In [33]:
# Função para validação cruzada do modelo LogisticRegression
def validacao_cruzada_sk(classificador, 
                         X_data, 
                         y_data, 
                         metrica1=precision_score,
                         metrica2=recall_score,
                         metrica3=f1_score,
                         metrica4=roc_auc_score,
                         num_folds=5, 
                         print_info=False, 
                         nome_metrica1='Precision',
                         nome_metrica2='Recall',
                         nome_metrica3='F1 Score',
                         nome_metrica4='AUC-ROC'):
    
    lista_metrica1_treino = []
    lista_metrica2_treino = []
    lista_metrica3_treino = []
    lista_metrica4_treino = []
    lista_metrica1_validacao = []
    lista_metrica2_validacao = []
    lista_metrica3_validacao = []
    lista_metrica4_validacao = []
    
    kf = KFold(n_splits = num_folds)
    for train_index, val_index in kf.split(X_data, y_data):
        Xtrain_folds = X_data[train_index]
        ytrain_folds = y_data[train_index]
        Xval_fold = X_data[val_index]
        yval_fold = y_data[val_index]
        classificador.fit(Xtrain_folds, ytrain_folds)
        
        pred_treino = classificador.predict(Xtrain_folds)
        pred_treino_proba = classificador.predict_proba(Xtrain_folds)[:, 1]
        
        pred_validacao = classificador.predict(Xval_fold)
        pred_validacao_proba = classificador.predict_proba(Xval_fold)[:, 1]
        
        # Guardando valores nas listas
        lista_metrica1_treino.append(metrica1(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica1_validacao.append(metrica1(y_pred = pred_validacao, y_true = yval_fold))
        
        lista_metrica2_treino.append(metrica2(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica2_validacao.append(metrica2(y_pred = pred_validacao, y_true = yval_fold))
        
        lista_metrica3_treino.append(metrica3(y_pred = pred_treino, y_true = ytrain_folds))
        lista_metrica3_validacao.append(metrica3(y_pred = pred_validacao, y_true = yval_fold))
        
        lista_metrica4_treino.append(metrica4(y_score = pred_treino_proba, y_true = ytrain_folds))
        lista_metrica4_validacao.append(metrica4(y_score = pred_validacao_proba, y_true = yval_fold))
        
    if print_info:
        print("Métrica: " + nome_metrica1)
        print('média treino:', np.mean(lista_metrica1_treino))
        print('média validação:', np.mean(lista_metrica1_validacao), end='\n\n')
        
        print("Métrica: " + nome_metrica2)
        print('média treino:', np.mean(lista_metrica2_treino))
        print('média validação:', np.mean(lista_metrica2_validacao), end='\n\n')
        
        print("Métrica: " + nome_metrica3)
        print('média treino:', np.mean(lista_metrica3_treino))
        print('média validação:', np.mean(lista_metrica3_validacao), end='\n\n')
        
        print("Métrica: " + nome_metrica4)
        print('média treino:', np.mean(lista_metrica4_treino))
        print('média validação:', np.mean(lista_metrica4_validacao), end='\n\n')
    
    lista_metrica1_treino = np.mean(lista_metrica1_treino)
    lista_metrica2_treino = np.mean(lista_metrica2_treino)
    lista_metrica3_treino = np.mean(lista_metrica3_treino)
    lista_metrica4_treino = np.mean(lista_metrica4_treino)
    
    lista_metrica1_validacao = np.mean(lista_metrica1_validacao)
    lista_metrica2_validacao = np.mean(lista_metrica2_validacao)
    lista_metrica3_validacao = np.mean(lista_metrica3_validacao)
    lista_metrica4_validacao = np.mean(lista_metrica4_validacao)
    
    df_ = pd.DataFrame(np.c_[lista_metrica1_treino, lista_metrica1_validacao, lista_metrica2_treino, lista_metrica2_validacao, lista_metrica3_treino, lista_metrica3_validacao, lista_metrica4_treino, lista_metrica4_validacao],
                      columns=['Precision (T)', 'Precision (V)', 'Recall (T)', 'Recall (V)', 'F1 (T)', 'F1 (V)', 'ROC-AUC (T)', 'ROC-AUC (V)'])
    if print_info:
        display(df_)
    
    return df_

In [34]:
df_proc_LRSL = validacao_cruzada_sk(classificador = LogisticRegression(),
                                                    X_data = Xtrain,
                                                    y_data = ytrain,
                                                    num_folds = 5,
                                                    print_info=print_info)

df_proc_LRSL

,Precision (T),Precision (V),Recall (T),Recall (V),F1 (T),F1 (V),ROC-AUC (T),ROC-AUC (V)
0,0.814485,0.814666,0.840938,0.841699,0.827498,0.827783,0.895884,0.895565


Mais uma vez, nota-se que a diferença entre os valores do F1 score são mínimos para o modelo LogisticRefression e regLogistica, sendo eles iguais a 0.8277 e 0.8269, respectivamente.

Buscando resultados melhores, a validação cruzada para o modelo implementado manualmente será feita, porém, considerando uma faixa mais estreita.

In [35]:
# Início da contagem do tempo de execução
t0 = time.time()

# Imprimir informações durante a execução
print_info = False

# Listas para armazenar métricas, learning_rate, num_steps e limiar
df_metricas = []
df_lr = []
df_ns = []
df_limiar = []

# Contador
i = 0

# Faixa de valores que serão testados
learningRate = np.linspace(0.02, 0.03, 100)
steps = np.arange(50, 110, 10)
limiares = np.arange(0.45, 0.56, 0.01)

for step in steps:
    for lr in learningRate:
        for limiar in limiares:
            dfb = validacao_cruzada_rl(classificador = regLogistica(learning_rate = lr, num_steps = step, limiar = limiar),
                                       X_data = Xtrain,
                                       y_data = ytrain,
                                       num_folds = 5,
                                       print_info=print_info)
            if (i == 0):
                df_metricas = dfb.copy()
            else:
                df_metricas.loc[i] = dfb.loc[:].values[0]
            
            df_lr.append(lr)
            df_ns.append(step)
            df_limiar.append(limiar)
            i += 1
            
nomes_colunas = ['Num. Passos', 'Learning Rate', 'Threshold']
nomes_colunas.extend(df_metricas.columns)

df_proc = pd.DataFrame(np.c_[df_ns, df_lr, df_limiar, df_metricas], columns=nomes_colunas)

f1_max = np.max(df_proc['F1 (V)'].values)
index_max = df_proc.index[df_proc['F1 (V)'] == f1_max][0]

# Fim da contagem do tempo de execução
t1 = time.time()
dt = t1 - t0

print('Tempo de execução: %d segundos' %(dt), end='\n\n')

highlight = lambda x:['background: lightgreen' if x['F1 (V)'] == f1_max else '' for i in x]
df_proc[index_max - 2 : index_max + 3].style.apply(highlight, axis=1)

Tempo de execução: 1341 segundos



,Num. Passos,Learning Rate,Threshold,Precision (T),Precision (V),Recall (T),Recall (V),F1 (T),F1 (V),ROC-AUC (T),ROC-AUC (V)
691,50.000000,0.025758,0.520000,0.820688,0.820368,0.820553,0.820615,0.820433,0.820160,0.882194,0.882641
692,50.000000,0.025758,0.530000,0.815815,0.815403,0.826992,0.830693,0.821052,0.822459,0.875356,0.875153
693,50.000000,0.025758,0.540000,0.812863,0.814228,0.842147,0.842475,0.827243,0.827911,0.872673,0.872797
694,50.000000,0.025758,0.550000,0.812316,0.811083,0.800849,0.801843,0.805817,0.805685,0.872636,0.874769
695,50.000000,0.025758,0.560000,0.801275,0.802456,0.844939,0.843978,0.820382,0.820055,0.881028,0.879580


Com o estreitamento da faixa de teste dos valores de num_steps, learning_rate e limiar, obteve-se um valor superior aos casos anteriores, porém, com uma diferença de apenas 0,06% para o melhor caso. Além disso, nota-se que há um pequeno aumento para a métrica ROC-AUC score também.

In [38]:
reg_log_m = regLogistica(learning_rate = 0.025758, num_steps = 50, limiar = 0.54)
reg_log_m.fit(Xtrain, ytrain)
ypred = reg_log_m.predict(Xtest)
f1_test = f1_score(y_true = ytest, y_pred = ypred)

Aplicando o modelo escolhido aos dados de teste, obtemos o seguinte valor de F1:

In [39]:
print('O valor de F1 score para os dados de teste é igual a %s.' %(str(f1_test)))

O valor de F1 score para os dados de teste é igual a 0.8327948303715671.
